# 进阶篇：样式化—StyleFrame

**DataFrames导出到样式化的Excel文件从未如此容易**

**StyleFrame**是一个封装panda和openpyxl的库，允许在Excel中轻松设置数据帧的样式。

## 理论基础

- Pandas的DataFrame非常棒。

- 处理大量数据并不容易，DataFrame可以帮助我们以最佳方式进行管理。

- 有很多种方式可以显示输出，其中之一就是excel文件。

- Excel文件易于理解，可以离线查看，可以通过电子邮件发送，而且大部分人都熟悉它。

这就是为什么很多时候我们会选择excel文件作为输出。

**StyleFrame包允许我们以类似于DataFrameapi的方式设计数据上的excel文件。**

**它为我们省去了使用excel工作簿的麻烦，也省去了将其与存储在DataFrame中的数据进行匹配的痛苦。**

## 安装

> pip install styleframe

## 基础

**Styler**：
> Styler类表示单元格的样式。



**utils**：
> utils模块包含常用样式元素的辅助类，如数字和日期格式、颜色和边框类型。



**Container**：
> Container类表示一个单元格，一个值/样式对。



**StyleFrame**：
> StyleFrame是您将拥有的主要交互点。它包装将要设置样式的DataFrame对象。

In [1]:
# 引入依赖包
import pandas as pd
from styleframe import StyleFrame, Styler, utils   

## 入门案例

In [2]:
df = pd.DataFrame({
    'Time': [1.496728e+09, 1.496728e+09, 1.496728e+09, 1.496728e+09, 1.496728e+09],
    'Expect': ['Hey', 'how', 'are', 'you', 'today?'],
    'Actual': ['Hello', 'how', 'are', 'u', 'today?'],
    'Pass/Fail': ['Failed', 'Passed', 'Passed', 'Failed', 'Passed']
    },
    columns=['Time', 'Expect', 'Actual', 'Pass/Fail'])
df

,Time,Expect,Actual,Pass/Fail
0,1.496728e+09,Hey,Hello,Failed
1,1.496728e+09,how,how,Passed
2,1.496728e+09,are,are,Passed
3,1.496728e+09,you,u,Failed
4,1.496728e+09,today?,today?,Passed


创建一个StyleFrame对象，参数包含一个Dataframe和默认的样式

In [3]:
# Create StyleFrame object that wrap our DataFrame and assign default style.
default_style = Styler(font=utils.fonts.aharoni, font_size=14)
sf = StyleFrame(df, styler_obj=default_style)

设置标题样式：标题加粗，字体大小为18

In [4]:
# Style the headers of the table
header_style = Styler(bold=True, font_size=18)
sf.apply_headers_style(styler_obj=header_style)

在测试结果为passed的单元格设置背景颜色为绿色

In [5]:
# Set the background color to green where the test marked as 'passed'
passed_style = Styler(bg_color=utils.colors.green, font_color=utils.colors.white)
sf.apply_style_by_indexes(indexes_to_style=sf[sf['Pass/Fail'] == 'Passed'],
                          cols_to_style='Pass/Fail',
                          styler_obj=passed_style,
                          overwrite_default_style=False)

在测试结果为passed的单元格设置背景颜色为绿色

In [6]:
# Set the background color to red where the test marked as 'failed'
failed_style = Styler(bg_color=utils.colors.red, font_color=utils.colors.white)
sf.apply_style_by_indexes(indexes_to_style=sf[sf['Pass/Fail'] == 'Failed'],
                          cols_to_style='Pass/Fail',
                          styler_obj=failed_style,
                          overwrite_default_style=False)

In [7]:
writer = sf.to_excel('data/styleframe测试数据导出.xlsx',
                     # Add filters in row 0 to each column.为每一列添加筛选
                     row_to_add_filters=0, 
                     # Freeze the columns before column 'A' (=None)
                     # and rows above '2' (=1).冻结窗口
                     columns_and_rows_to_freeze='A2')

writer.close()

![image.png](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/pandas/styleframe%E6%B5%8B%E8%AF%95%E6%95%B0%E6%8D%AE%E5%AF%BC%E5%87%BA.png)

## 进阶案例

First, let us create a DataFrame that contains data we would like to export to an .xlsx file

首先，让我们创建一个dataframe，包含我们想要导出excel的数据

In [8]:
# 引入依赖包
from datetime import date
import pandas as pd

In [9]:
columns = ['Date', 'Col A', 'Col B', 'Col C', 'Percentage']
df = pd.DataFrame(data={'Date': [date(1995, 9, 5), date(1947, 11, 29), date(2000, 1, 15)],
                        'Col A': [1, 2004, -3],
                        'Col B': [15, 3, 116],
                        'Col C': [33, -6, 9],
                        'Percentage': [0.113, 0.504, 0.005]},
                  columns=columns)
df

,Date,Col A,Col B,Col C,Percentage
0,1995-09-05,1,15,33,0.113
1,1947-11-29,2004,3,-6,0.504
2,2000-01-15,-3,116,9,0.005


In [10]:
only_values_df = df[columns[1:-1]]

rows_max_value = only_values_df.idxmax(axis=1)

df['Sum'] = only_values_df.sum(axis=1)
df['Mean'] = only_values_df.mean(axis=1)

In [11]:
# Our DataFrame looks like this
df

,Date,Col A,Col B,Col C,Percentage,Sum,Mean
0,1995-09-05,1,15,33,0.113,49,16.333333
1,1947-11-29,2004,3,-6,0.504,2001,667.000000
2,2000-01-15,-3,116,9,0.005,122,40.666667


In [12]:
# Now, once we have the DataFrame ready, lets create a StyleFrame object
from styleframe import StyleFrame

sf = StyleFrame(df)
# it is also possible to directly initiate StyleFrame
"""
sf = StyleFrame({'Date': [date(1995, 9, 5), date(1947, 11, 29), date(2000, 1, 15)],
                 'Col A': [1, 2004, -3],
                 'Col B': [15, 3, 116],
                 'Col C': [33, -6, 9],
                 'Percentage': [0.113, 0.504, 0.005],
                 'Sum': [49, 2001, 122],
                 'Mean': [16.333333, 667.000000, 40.666667]})
"""

"\nsf = StyleFrame({'Date': [date(1995, 9, 5), date(1947, 11, 29), date(2000, 1, 15)],\n                 'Col A': [1, 2004, -3],\n                 'Col B': [15, 3, 116],\n                 'Col C': [33, -6, 9],\n                 'Percentage': [0.113, 0.504, 0.005],\n                 'Sum': [49, 2001, 122],\n                 'Mean': [16.333333, 667.000000, 40.666667]})\n"

The StyleFrame object will auto-adjust the columns width and the rows height but they can be changed manually

StyleFrame可以自动调整列宽和行高，但是它们也可以被手动修改

In [13]:
sf.set_column_width_dict(col_width_dict={
    ('Col A', 'Col B', 'Col C'): 15.3,
    ('Sum', 'Mean'): 30,
    ('Percentage', ): 12
})

# excel rows starts from 1
# row number 1 is the headers
# len of StyleFrame (same as DataFrame) does not count the headers row
all_rows = sf.row_indexes
sf.set_row_height_dict(row_height_dict={
    all_rows[0]: 45,  # headers row
    all_rows[1:]: 25
})

Applying number formats
数字格式应用

In [14]:
from styleframe import Styler, utils


sf.apply_column_style(cols_to_style='Date',
                      styler_obj=Styler(date_format=utils.number_formats.date,
                                        font=utils.fonts.calibri,
                                        bold=True))

sf.apply_column_style(cols_to_style='Percentage',
                      styler_obj=Styler(number_format=utils.number_formats.percent))

sf.apply_column_style(cols_to_style=['Col A', 'Col B', 'Col C'],
                      styler_obj=Styler(number_format=utils.number_formats.thousands_comma_sep))                     


Next, let's change the background color of the maximum values to red and the font to white

接下来，我们将每列的最大值的背景颜色改为红色，字体颜色改为白色

we will also protect those cells and prevent the ability to change their value

我们也将设置表格为保护模式，防止他们的值被修改

In [15]:
style = Styler(bg_color=utils.colors.red,
               bold=True,
               font_color=utils.colors.white,
               protection=True,
               underline=utils.underline.double,
               number_format=utils.number_formats.thousands_comma_sep)
        
for row_index, col_name in rows_max_value.iteritems():
    sf[col_name][row_index].style = style

And change the font and the font size of Sum and Mean columns

修改Sum和Mean列的字体和字体大小

In [16]:
sf.apply_column_style(cols_to_style=['Sum', 'Mean'],
                      style_header=True,
                      styler_obj=Styler(font_color='#40B5BF',
                                        font_size=18,
                                        bold=True))

Change the background of all rows where the date is after 14/1/2000 to green

修改日期大于14/1/2000的所有行背景颜色为绿色

In [17]:
sf.apply_style_by_indexes(indexes_to_style=sf[sf['Date'] > date(2000, 1, 14)],
                          cols_to_style='Date',
                          styler_obj=Styler(bg_color=utils.colors.green,
                                            date_format=utils.number_formats.date,
                                            bold=True))

Finally, let's export to Excel but not before we use more of StyleFrame's features:

- Change the page writing side
- Freeze rows and columns
- Add filters to headers

In [18]:
ew = StyleFrame.ExcelWriter('data/sf教学.xlsx')
sf.to_excel(excel_writer=ew,
            sheet_name='1',
            right_to_left=False,
            columns_and_rows_to_freeze='B2', # will freeze the rows above 2 (=row 1 only) and columns that before column 'B' (=col A only)
            row_to_add_filters=0,
            allow_protection=True)

In [19]:
ew.close()

Adding another excel sheet

In [20]:
other_sheet_sf = StyleFrame({'Dates': [date(2016, 10, 20), date(2016, 10, 21), date(2016, 10, 22)]},
                            styler_obj=Styler(date_format=utils.number_formats.date))
                            
other_sheet_sf.to_excel(excel_writer=ew, sheet_name='2')

Don't forget to save

In [21]:
ew.close()

生成的文件 sheet1
![sheet1](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/pandas/Snipaste_2024-01-01_12-30-34.png)

sheet2

![sheet2](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/pandas/Snipaste_2024-01-01_12-32-05.png)

## [API Documentation](https://styleframe.readthedocs.io/en/latest/)

### styleframe
- StyleFrame
    - StyleFrame.ExcelWriter()
    - StyleFrame.add_color_scale_conditional_formatting()
    - StyleFrame.apply_column_style()
    - StyleFrame.apply_headers_style()
    - StyleFrame.apply_style_by_indexes()
    - StyleFrame.read_excel()
    - StyleFrame.read_excel_as_template()
    - StyleFrame.rename()
    - StyleFrame.set_column_width()
    - StyleFrame.set_column_width_dict()
    - StyleFrame.set_row_height()
    - StyleFrame.set_row_height_dict()
    - StyleFrame.style_alternate_rows()
    - StyleFrame.to_excel()


### styler
-  Styler
    - Styler.combine()


### utils
-  number_formats
    - number_formats.decimal_with_num_of_digits()

-  colors
-  fonts
-  borders
-  border_locations
-  horizontal_alignments
-  vertical_alignments
-  underline
-  fill_pattern_types
-  conditional_formatting_types